In [131]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
train_data = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
X = train_data.drop(columns=["Transported","PassengerId"])
y = train_data["Transported"]
test_data = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


# Data Cleaning & Preprocessing:


In [132]:
# Espansione della feature Cabin: 
new = X["Cabin"].str.split("/", n=-1, expand=True)
X["Deck"] = new[0] # Aggiungo la colonna Deck
X["Num"] = new[1]  # Aggiungo la colonna Num
X["Side"] = new[2] # Aggiungo la colonna Side
X.drop(columns=["Cabin"], inplace=True)
X.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Deck,Num,Side
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,B,0,P
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,F,0,S
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,A,0,S
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,A,0,S
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,F,1,S


In [133]:
# Verifica della percentuale di valori mancanti per ogni feature:
percentuali_nan = X.isnull().sum()/len(X)*100
print(percentuali_nan)

features = ["HomePlanet","CryoSleep","Destination","Age","VIP","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","Deck","Num","Side"]
X = X[features]
X = pd.get_dummies(X,columns=["HomePlanet","Destination","Deck","Side"])

# Gestione dei valori mancanti:
X.dropna(inplace=True) # Remove rows

HomePlanet      2.312205
CryoSleep       2.496261
Destination     2.093639
Age             2.059128
VIP             2.335212
RoomService     2.082135
FoodCourt       2.105142
ShoppingMall    2.392730
Spa             2.105142
VRDeck          2.162660
Name            2.300702
Deck            2.289198
Num             2.289198
Side            2.289198
dtype: float64


In [134]:
# Standardizzazione della matrice X:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
nd_X_scaled = scaler.fit_transform(X)
pd_X_scaled = pd.DataFrame(nd_X_scaled, columns=X.columns)
pd_X_scaled.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Num,HomePlanet_Earth,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_P,Side_S
0,-0.733253,0.695256,-0.157277,-0.349048,-0.286200,-0.284059,-0.273687,-0.271714,-1.169723,-1.057554,...,-0.179029,3.101724,-0.309155,-0.243982,-0.342564,-0.698962,-0.650871,-0.023769,1.014217,-1.014217
1,-0.733253,-0.340058,-0.157277,-0.179371,-0.280744,-0.244240,0.204749,-0.232327,-1.169723,0.945578,...,-0.179029,-0.322401,-0.309155,-0.243982,-0.342564,1.430693,-0.650871,-0.023769,-0.985982,0.985982
2,-0.733253,2.006654,6.358215,-0.282111,1.881943,-0.284059,5.578215,-0.227852,-1.169723,-1.057554,...,5.585696,-0.322401,-0.309155,-0.243982,-0.342564,-0.698962,-0.650871,-0.023769,-0.985982,0.985982
3,-0.733253,0.281131,-0.157277,-0.349048,0.491688,0.306849,2.627428,-0.098950,-1.169723,-1.057554,...,5.585696,-0.322401,-0.309155,-0.243982,-0.342564,-0.698962,-0.650871,-0.023769,-0.985982,0.985982
4,-0.733253,-0.892225,-0.157277,0.122623,-0.243759,-0.043555,0.218692,-0.269924,-1.167774,0.945578,...,-0.179029,-0.322401,-0.309155,-0.243982,-0.342564,1.430693,-0.650871,-0.023769,-0.985982,0.985982


In [135]:
# Principal Component Analysis:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(nd_X_scaled)
print(X_pca)
print(f"Explained variance: {pca.explained_variance_ratio_}")
print(f"Cumulative: {np.cumsum(pca.explained_variance_ratio_)}")

[[ 2.28334219  0.12539703]
 [-0.76120863  0.53327652]
 [ 5.26064284  0.43249829]
 ...
 [-1.89402928 -0.49622283]
 [ 2.65521486 -0.65403734]
 [ 2.00213586  0.64334931]]
Explained variance: [0.14894334 0.10180119]
Cumulative: [0.14894334 0.25074453]


In [136]:
# Plot della PCA:
"""
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.scatter(nd_X_scaled[:, 0], nd_X_scaled[:, 1], c=y, cmap='coolwarm', edgecolor='k')
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.title("Original Data (First Two Features)")
plt.colorbar(label="Diagnosis")
plt.show()

plt.figure(figsize=(8,6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='coolwarm', edgecolor='k')
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.title("PCA Transformed Data")
plt.colorbar(label="Diagnosis")
plt.show()
"""

'\nimport matplotlib.pyplot as plt\n\nplt.figure(figsize=(8,6))\nplt.scatter(nd_X_scaled[:, 0], nd_X_scaled[:, 1], c=y, cmap=\'coolwarm\', edgecolor=\'k\')\nplt.xlabel("Feature 1")\nplt.ylabel("Feature 2")\nplt.title("Original Data (First Two Features)")\nplt.colorbar(label="Diagnosis")\nplt.show()\n\nplt.figure(figsize=(8,6))\nplt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap=\'coolwarm\', edgecolor=\'k\')\nplt.xlabel("Principal Component 1")\nplt.ylabel("Principal Component 2")\nplt.title("PCA Transformed Data")\nplt.colorbar(label="Diagnosis")\nplt.show()\n'